In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.layers import LeakyReLU

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=['EIN','NAME'])
application_df

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [3]:
# Look at APPLICATION_TYPE value counts for binning
application_counts = application_df.APPLICATION_TYPE.value_counts()

# Determine which values to replace if counts are less than ...?
replace_application = list(application_counts[application_counts < 500].index)

# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
application_df.APPLICATION_TYPE.value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [4]:
# Look at CLASSIFICATION value counts for binning
classification_counts = application_df.CLASSIFICATION.value_counts()

# Determine which values to replace if counts are less than ..?
replace_class = list(classification_counts[classification_counts < 1000].index)

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [5]:
# Generate our categorical variable lists
application_cat = application_df.dtypes[application_df.dtypes == "object"].index.tolist()

# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)
encode_df.head()

,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [6]:
# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encode_df ,left_index=True, right_index=True)
application_df = application_df.drop(columns=application_cat)
application_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [7]:
# Split our preprocessed data into our features and target arrays
X = application_df.drop(['IS_SUCCESSFUL'],1).values
y = application_df['IS_SUCCESSFUL'].values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=42)

In [8]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [9]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_features = len(X_train[0])
hidden_nodes_1 = 80
hidden_nodes_2 = 30

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_1, activation=keras.layers.LeakyReLU(alpha=0.01), input_dim=input_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_2, activation='relu', input_dim=input_features))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 80)                3520      
                                                                 
 dense_1 (Dense)             (None, 30)                2430      
                                                                 
 dense_2 (Dense)             (None, 1)                 31        
                                                                 
Total params: 5,981
Trainable params: 5,981
Non-trainable params: 0
_________________________________________________________________


In [10]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [11]:
# Create a callback that saves the model's weights every 5 epochs
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

# Create a callback that saves the model's weights every epoch
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=5)

In [12]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 1s 772us/step - loss: 0.5715 - accuracy: 0.7198
Epoch 2/100
804/804 [==============================] - 1s 754us/step - loss: 0.5555 - accuracy: 0.7271
Epoch 3/100
804/804 [==============================] - 1s 749us/step - loss: 0.5522 - accuracy: 0.7297
Epoch 4/100
804/804 [==============================] - 1s 745us/step - loss: 0.5511 - accuracy: 0.7308
Epoch 5/100
804/804 [==============================] - 1s 796us/step - loss: 0.5498 - accuracy: 0.7310
Epoch 6/100
804/804 [==============================] - 1s 754us/step - loss: 0.5487 - accuracy: 0.7318
Epoch 7/100
804/804 [==============================] - 1s 764us/step - loss: 0.5480 - accuracy: 0.7312
Epoch 8/100
804/804 [==============================] - 1s 879us/step - loss: 0.5475 - accuracy: 0.7334
Epoch 9/100
804/804 [==============================] - 1s 864us/step - loss: 0.5464 - accuracy: 0.7335
Epoch 10/100
804/804 [==============================] - 1s 809us/step - l

804/804 [==============================] - 1s 792us/step - loss: 0.5351 - accuracy: 0.7403
Epoch 80/100
804/804 [==============================] - 1s 767us/step - loss: 0.5350 - accuracy: 0.7394
Epoch 81/100
804/804 [==============================] - 1s 792us/step - loss: 0.5350 - accuracy: 0.7400
Epoch 82/100
804/804 [==============================] - 1s 766us/step - loss: 0.5352 - accuracy: 0.7405
Epoch 83/100
804/804 [==============================] - 1s 774us/step - loss: 0.5351 - accuracy: 0.7391
Epoch 84/100
804/804 [==============================] - 1s 806us/step - loss: 0.5351 - accuracy: 0.7404
Epoch 85/100
804/804 [==============================] - 1s 776us/step - loss: 0.5349 - accuracy: 0.7407
Epoch 86/100
804/804 [==============================] - 1s 781us/step - loss: 0.5350 - accuracy: 0.7396
Epoch 87/100
804/804 [==============================] - 1s 782us/step - loss: 0.5348 - accuracy: 0.7392
Epoch 88/100
804/804 [==============================] - 1s 819us/step - loss:

In [13]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5629 - accuracy: 0.7285 - 252ms/epoch - 940us/step
Loss: 0.5628588795661926, Accuracy: 0.7285131216049194


In [14]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_OptimizationAttempt1.h5")

In [15]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_features = len(X_train[0])
hidden_nodes_1 = 100
hidden_nodes_2 = 50

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_1, activation=keras.layers.LeakyReLU(alpha=0.01), input_dim=input_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_2, activation='relu', input_dim=input_features))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 100)               4400      
                                                                 
 dense_4 (Dense)             (None, 50)                5050      
                                                                 
 dense_5 (Dense)             (None, 1)                 51        
                                                                 
Total params: 9,501
Trainable params: 9,501
Non-trainable params: 0
_________________________________________________________________


In [16]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [17]:
# Create a callback that saves the model's weights every 5 epochs
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

# Create a callback that saves the model's weights every epoch
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=5)

In [18]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 1s 908us/step - loss: 0.5711 - accuracy: 0.7232
Epoch 2/100
804/804 [==============================] - 1s 875us/step - loss: 0.5557 - accuracy: 0.7276
Epoch 3/100
804/804 [==============================] - 1s 932us/step - loss: 0.5525 - accuracy: 0.7309
Epoch 4/100
804/804 [==============================] - 1s 840us/step - loss: 0.5505 - accuracy: 0.7296
Epoch 5/100
804/804 [==============================] - 1s 810us/step - loss: 0.5492 - accuracy: 0.7303
Epoch 6/100
804/804 [==============================] - 1s 803us/step - loss: 0.5484 - accuracy: 0.7323
Epoch 7/100
804/804 [==============================] - 1s 798us/step - loss: 0.5472 - accuracy: 0.7331
Epoch 8/100
804/804 [==============================] - 1s 953us/step - loss: 0.5461 - accuracy: 0.7317
Epoch 9/100
804/804 [==============================] - 1s 875us/step - loss: 0.5466 - accuracy: 0.7330
Epoch 10/100
804/804 [==============================] - 1s 949us/step - l

804/804 [==============================] - 1s 784us/step - loss: 0.5343 - accuracy: 0.7401
Epoch 80/100
804/804 [==============================] - 1s 898us/step - loss: 0.5340 - accuracy: 0.7396
Epoch 81/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5341 - accuracy: 0.7397
Epoch 82/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5346 - accuracy: 0.7401
Epoch 83/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5341 - accuracy: 0.7398
Epoch 84/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5348 - accuracy: 0.7398
Epoch 85/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5343 - accuracy: 0.7394
Epoch 86/100
804/804 [==============================] - 1s 982us/step - loss: 0.5341 - accuracy: 0.7402
Epoch 87/100
804/804 [==============================] - 1s 862us/step - loss: 0.5337 - accuracy: 0.7398
Epoch 88/100
804/804 [==============================] - 1s 818us/step - loss: 0.5334 - 

In [19]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5662 - accuracy: 0.7269 - 221ms/epoch - 824us/step
Loss: 0.5661759376525879, Accuracy: 0.7268804907798767


In [20]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_OptimizationAttempt2.h5")

In [21]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_features = len(X_train[0])
hidden_nodes_1 = 100
hidden_nodes_2 = 50
hidden_nodes_3 = 20

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_1, activation='relu', input_dim=input_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_2, activation='relu', input_dim=input_features))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_3, activation='relu', input_dim=input_features))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 100)               4400      
                                                                 
 dense_7 (Dense)             (None, 50)                5050      
                                                                 
 dense_8 (Dense)             (None, 20)                1020      
                                                                 
 dense_9 (Dense)             (None, 1)                 21        
                                                                 
Total params: 10,491
Trainable params: 10,491
Non-trainable params: 0
_________________________________________________________________


In [22]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Create a callback that saves the model's weights every 5 epochs
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

# Create a callback that saves the model's weights every epoch
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=5)

# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=150)

Epoch 1/150
804/804 [==============================] - 1s 919us/step - loss: 0.5670 - accuracy: 0.7211
Epoch 2/150
804/804 [==============================] - 1s 923us/step - loss: 0.5549 - accuracy: 0.7282
Epoch 3/150
804/804 [==============================] - 1s 915us/step - loss: 0.5507 - accuracy: 0.7297
Epoch 4/150
804/804 [==============================] - 1s 906us/step - loss: 0.5495 - accuracy: 0.7304
Epoch 5/150
804/804 [==============================] - 1s 916us/step - loss: 0.5486 - accuracy: 0.7316
Epoch 6/150
804/804 [==============================] - 1s 918us/step - loss: 0.5476 - accuracy: 0.7331
Epoch 7/150
804/804 [==============================] - 1s 911us/step - loss: 0.5473 - accuracy: 0.7336
Epoch 8/150
804/804 [==============================] - 1s 903us/step - loss: 0.5462 - accuracy: 0.7345
Epoch 9/150
804/804 [==============================] - 1s 914us/step - loss: 0.5459 - accuracy: 0.7340
Epoch 10/150
804/804 [==============================] - 1s 917us/step - l

804/804 [==============================] - 1s 892us/step - loss: 0.5343 - accuracy: 0.7397
Epoch 80/150
804/804 [==============================] - 1s 888us/step - loss: 0.5338 - accuracy: 0.7402
Epoch 81/150
804/804 [==============================] - 1s 888us/step - loss: 0.5337 - accuracy: 0.7397
Epoch 82/150
804/804 [==============================] - 1s 885us/step - loss: 0.5337 - accuracy: 0.7409
Epoch 83/150
804/804 [==============================] - 1s 884us/step - loss: 0.5331 - accuracy: 0.7404
Epoch 84/150
804/804 [==============================] - 1s 889us/step - loss: 0.5332 - accuracy: 0.7406
Epoch 85/150
804/804 [==============================] - 1s 905us/step - loss: 0.5332 - accuracy: 0.7405
Epoch 86/150
804/804 [==============================] - 1s 888us/step - loss: 0.5331 - accuracy: 0.7392
Epoch 87/150
804/804 [==============================] - 1s 890us/step - loss: 0.5332 - accuracy: 0.7402
Epoch 88/150
804/804 [==============================] - 1s 889us/step - loss:

In [23]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5868 - accuracy: 0.7285 - 233ms/epoch - 870us/step
Loss: 0.5867565870285034, Accuracy: 0.7285131216049194


In [24]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_OptimizationAttempt3.h5")

In [25]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_Optimization.h5")